# Notebook 5 : Test de l'API de Scoring Crédit

Ce notebook teste l'API de scoring crédit déployée sur Railway avec les 2 clients spécifiques analysés dans le Notebook 3 : ID 243023 (haut risque) et ID 288878 (faible risque).

## Objectifs

- Tester la connectivité de l'API (/health)
- Valider le fonctionnement de l'endpoint de prédiction (/predict) avec les 2 clients extrêmes
- Vérifier la cohérence des décisions avec les résultats du Notebook 3
- Démontrer la reproductibilité des prédictions

# Impots et configuration

In [1]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime

# Configuration de l'API
API_BASE_URL = "https://reliable-vitality-production.up.railway.app"
HEALTH_URL = f"{API_BASE_URL}/health"
PREDICT_URL = f"{API_BASE_URL}/predict"

print(f"URL de l'API: {API_BASE_URL}")
print(f"Date du test: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

URL de l'API: https://reliable-vitality-production.up.railway.app
Date du test: 2025-06-12 11:18:58


# Extraction des 2 clients extrêmes du notebook 3 (SHAP local)

In [2]:
# IDs des clients extrêmes identifiés dans le Notebook 3
client_haut_risque_id = 243023
client_faible_risque_id = 288878

# Charger les données preprocessées
try:
    # Essayer différents chemins possibles
    possible_paths = [
        'data/processed/train_preprocessed.csv',
        '../data/processed/train_preprocessed.csv',
        'train_preprocessed.csv'
    ]
    
    data = None
    for path in possible_paths:
        try:
            data = pd.read_csv(path, index_col='SK_ID_CURR')
            print(f"Données chargées depuis: {path}")
            break
        except FileNotFoundError:
            continue
    
    if data is None:
        raise FileNotFoundError("Aucun fichier train_preprocessed.csv trouvé dans les chemins testés")
        
except Exception as e:
    print(f" Erreur lors du chargement: {e}")
    print("Chemins testés:")
    for path in possible_paths:
        print(f"  - {path}")
    # Arrêter l'exécution si pas de données
    raise

# Supprimer TARGET présent
if 'TARGET' in data.columns:
    data = data.drop('TARGET', axis=1)

# Vérifier que les clients sont présents
clients_presents = []
for client_id in [client_haut_risque_id, client_faible_risque_id]:
    if client_id in data.index:
        clients_presents.append(client_id)
        print(f" Client {client_id} trouvé dans les données")
    else:
        print(f" Client {client_id} non trouvé dans les données")

if len(clients_presents) != 2:
    print(" Impossible de continuer sans les 2 clients requis")
    exit()

# Fonction de nettoyage des données pour JSON
def clean_data_for_json(data_dict):
    """Nettoie les données pour les rendre compatibles JSON"""
    cleaned = {}
    for key, value in data_dict.items():
        if pd.isna(value):
            cleaned[key] = 0.0
        elif np.isinf(value):
            cleaned[key] = 0.0
        elif isinstance(value, (np.integer, np.floating)):
            cleaned[key] = float(value)
        else:
            cleaned[key] = value
    return cleaned

# Extraire les 2 clients extrêmes
client_haut_risque_raw = data.loc[client_haut_risque_id].to_dict()
client_faible_risque_raw = data.loc[client_faible_risque_id].to_dict()

# Nettoyer les données pour JSON
client_haut_risque = clean_data_for_json(client_haut_risque_raw)
client_faible_risque = clean_data_for_json(client_faible_risque_raw)

Données chargées depuis: ../data/processed/train_preprocessed.csv
 Client 243023 trouvé dans les données
 Client 288878 trouvé dans les données


# Endpoint /health

In [3]:
try:
    response = requests.get(HEALTH_URL, timeout=10)
    print(f"Status: {'Réussi' if response.status_code == 200 else 'Echec'}")
    print(f"Code de réponse: {response.status_code}")
    
    if response.status_code == 200:
        health_data = response.json()
        print("\nContenu de la réponse:")
        print(json.dumps(health_data, indent=2, ensure_ascii=False))
        
        # Validations spécifiques
        validations = [
            (health_data.get("status") == "ONLINE", "Service en ligne"),
            (health_data.get("features_count") == 234, "Nombre de features correct (234)"),
            (health_data.get("model") == "LightGBM Optimized", "Modèle correct"),
            ("threshold" in health_data, "Seuil optimal présent")
        ]
        
        for validation, message in validations:
            print(f"{message} {'OK' if validation else 'KO'}")
            
except Exception as e:
    print(f"Erreur lors du test /health: {e}")

Status: Réussi
Code de réponse: 200

Contenu de la réponse:
{
  "features_count": 234,
  "model": "LightGBM Optimized",
  "platform": "Railway",
  "service": "Credit Scoring API",
  "status": "ONLINE",
  "threshold": 0.09909090909090908,
  "version": "RAILWAY_V2"
}
Service en ligne OK
Nombre de features correct (234) OK
Modèle correct OK
Seuil optimal présent OK


# Prédiction des 2 clients

In [4]:
def tester_prediction(client_data, nom_profil):
    """Teste une prédiction et affiche le statut et la décision"""
    try:
        response = requests.post(
            PREDICT_URL, 
            json=client_data,
            headers={"Content-Type": "application/json"},
            timeout=15
        )
        
        print(f"\n{nom_profil}")
        status = " Réussi" if response.status_code == 200 else "Echec"
        print(f"Status: {status}")
        
        if response.status_code == 200:
            prediction = response.json()
            decision = prediction.get('decision', 'N/A')
            print(f"Décision: {decision}")
            return prediction
        else:
            print(f"Décision: Erreur (Code {response.status_code})")
            return None
            
    except Exception as e:
        print(f"Status: Echec")
        print(f"Décision: Erreur ({e})")
        return None

# Tester les 2 clients
prediction_haut_risque = tester_prediction(
    client_haut_risque, 
    f"Client Haut Risque REFUSE DANS SHAP (ID: {client_haut_risque_id})"
)

prediction_faible_risque = tester_prediction(
    client_faible_risque, 
    f"Client Faible Risque ACCORDE DANS SHAP (ID: {client_faible_risque_id})"
)


Client Haut Risque REFUSE DANS SHAP (ID: 243023)
Status:  Réussi
Décision: REFUSE

Client Faible Risque ACCORDE DANS SHAP (ID: 288878)
Status:  Réussi
Décision: ACCORDE
